<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

## loading data

In [1]:
# unzip the data from it living place into our current data_dir
!unzip ../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip

Archive:  ../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip
  inflating: Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv
   creating: __MACOSX/
  inflating: __MACOSX/._Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv  


In [2]:
# eww trash need to get rid of that real quick
!rm -r __MACOSX

In [4]:
# move the .csv into our current data dir
!mv ./*.csv ./data

## putting data in a df and some eda

In [21]:
# load into a pandas df and take a look at the data
import pandas as pd;
df = pd.read_csv("./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
print("df shape:", df.shape, "\n")
df.head()


df shape: (28332, 24) 



,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2017-03-02T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-08-31T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-12-06T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-03-28T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2017-04-22T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [22]:
# how many products are there being reviewed in this dataset?
df.asins.nunique()

65

In [23]:
# how many prime cats do we have products for?
df.primaryCategories.nunique()

9

In [24]:
# get a list of columns to look at
df.columns.to_list()

['id',
 'dateAdded',
 'dateUpdated',
 'name',
 'asins',
 'brand',
 'categories',
 'primaryCategories',
 'imageURLs',
 'keys',
 'manufacturer',
 'manufacturerNumber',
 'reviews.date',
 'reviews.dateSeen',
 'reviews.didPurchase',
 'reviews.doRecommend',
 'reviews.id',
 'reviews.numHelpful',
 'reviews.rating',
 'reviews.sourceURLs',
 'reviews.text',
 'reviews.title',
 'reviews.username',
 'sourceURLs']

In [25]:
# drop usless junk in the persuit of smaller impact to computation
drop_list = ['imageURLs',
             'id',
             'dateUpdated',
             'asins',
             'categories',
             'keys',
             'manufacturerNumber',
             'reviews.date',
             'reviews.dateSeen',
             'reviews.didPurchase',
             'reviews.doRecommend',
             'reviews.id',
             'reviews.numHelpful',
             'reviews.sourceURLs',
             'sourceURLs',
             'reviews.username']

df.drop(drop_list,axis=1,inplace=True)

In [26]:
# seems like the majority of cats are in electronics and health&beauty
# going to just use electronics since i have domain knoledge there
df.primaryCategories.value_counts(normalize=True)


Electronics                    0.493964
Health & Beauty                0.426055
Toys & Games,Electronics       0.059156
Office Supplies,Electronics    0.013624
Electronics,Media              0.006530
Office Supplies                0.000318
Animals & Pet Supplies         0.000212
Home & Garden                  0.000071
Electronics,Furniture          0.000071
Name: primaryCategories, dtype: float64

In [27]:
df_sm = df[df['primaryCategories'] == 'Electronics']


In [28]:
# cut about half of the size down which duh look at the normalized value_counts
df_sm.shape

(13995, 8)

In [42]:
# I wanna see what type of data im going to be 
# looking at to see how i have to clean it if i have to clean it
pd.set_option('display.max_colwidth', -1)
df_sm['reviews.text'].sample(10)

19527    Our 1 1/2 year old daughter loves this one and she is very delighted with this one                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
12575    I bought the Amazon Tap when it was on sale. I already had the Amazon Echo and the Dot. The Tap was a pleasant surprise. Speakers aren't as strong as the original Echo, but the sound quality of voice and music is more than acceptable. TheTap will do virtually anything that the more Expensive Echo can do  including hands-free voice commands. Plus, it's portable and h

## tokenization of reviews text

In [43]:
# ideally you only want to load this once.
import en_core_web_lg
nlp = en_core_web_lg.load()

In [52]:
def tokenize(df:pd.DataFrame,col_name:str):
    """Takes docs passed in df[col_name] and tokenizes based on lemmas
    Arguments:
    ----------
    df {pd.Dataframe} - the data frame holding the text data that needs to be tokenized

    col_name {str} - the column name for the dataframe column that has the text data
    Returns:
    ----------
    tokens {spacy.tokens.token.Token}
    """
    tokens = []
    for doc in nlp.pipe(df[col_name]):
        doc_tokens = []
        for token in doc:
            # strip stopwords pronounds and punctuation from tokens then strip leading ws
            if ((token.is_stop != True) and (token.pos_ != 'PRON')) and (token.is_punct !=True):
                doc_tokens.append(token.lemma_.strip())
        tokens.append(doc_tokens)
    return tokens
df_sm['tokens'] = tokenize(df_sm,'reviews.text')
# make sur ethat it looks ok or what i exspect a token
df_sm.tokens.iloc[0]

['great', 'case', 'place', 'husband', 'love', 'hold', 'lot', 'cd']

In [46]:
# unit test / sanity check
assert len(tokenize(df=df_sm, col_name='reviews.text')) == df_sm.shape[0]

## gensim LDA

In [48]:
import gensim
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [53]:
id2word = corpora.Dictionary(df_sm.tokens)

print("length of token keys-Unfiltered", len(id2word.keys()))

#filter out extreams in tokens
id2word.filter_extremes(no_below=5, no_above=.95)

print("length of token keys-filtered", len(id2word.keys()))

corpora = [id2word.doc2bow(doc) for doc in df_sm.tokens]

lda = LdaMulticore(corpus=corpora,
                  id2word=id2word,
                  num_topics=15, 
                  passes=50)

length of token keys-Unfiltered 6832
length of token keys-filtered 2247


In [49]:
import re
def clean_print_topics(lda:LdaMulticore):
    """Prints out the topics from an LDA in a nice pretty list for humans to read
    Arguments:
    ---------
    lda {LdaMulticore} - 
    """
    words = [re.findall('"([^"]*)"',t[1]) for t in lda.print_topics()]
    topics = [' '.join(t[0:5]) for t in words]
    for id, t in enumerate(topics): 
        print(f"------ Topic {id} ------")
        print(t, end="\n\n")
    return None

In [54]:
clean_print_topics(lda)

------ Topic 0 ------
battery life charge long tablet

------ Topic 1 ------
tablet buy good charge price

------ Topic 2 ------
8 Fire screen 7 Amazon

------ Topic 3 ------
kindle love fire buy old

------ Topic 4 ------
tablet easy use need great

------ Topic 5 ------
movie great watch tablet work

------ Topic 6 ------
great good tablet price product

------ Topic 7 ------
love buy old game year

------ Topic 8 ------
easy use light read case

------ Topic 9 ------
app tablet kid Amazon device

------ Topic 10 ------
tablet kid good get price

------ Topic 11 ------
child tablet purchase Amazon son

------ Topic 12 ------
sound speaker Alexa music great

------ Topic 13 ------
tablet Amazon app store good

------ Topic 14 ------
Kindle book read reader Fire



## dope visuals 

![https://i.redd.it/ih7217ioxwcx.jpg](https://i.redd.it/ih7217ioxwcx.jpg)

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling